## Problem 5

In [1]:
## For this question, fill in the TODO's 

In [2]:
!!pip install adapter-transformers==1.0.1
!git clone https://github.com/huggingface/transformers
!python transformers/utils/download_glue_data.py --tasks RTE

import dataclasses
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Dict, Optional

import numpy as np

import torch
from transformers import AutoTokenizer, EvalPrediction, GlueDataset, GlueDataTrainingArguments, AutoModelWithHeads, AdapterType, AutoConfig, AutoModelForSequenceClassification
from transformers import GlueDataTrainingArguments as DataTrainingArguments
from transformers import (
    Trainer,
    TrainingArguments,
    glue_compute_metrics,
    glue_tasks_num_labels,
    set_seed,
)

# Import statistics for mean and variance calculation
from statistics import *

model_name = "bert-base-uncased"# TODO 

# Refer to the notebook for training an adapter to write these. Set the number of epochs to 3, and learning rate to 5e-5. Rest of the hyperparameters can stay the same. 

data_args = GlueDataTrainingArguments(task_name="rte", data_dir= "./glue_data/RTE")# TODO 

training_args = TrainingArguments(
    logging_steps=50, 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=64, 
    save_steps=1000,
    evaluate_during_training=True,
    output_dir="./models/rte",
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    do_predict=True,
    learning_rate=5e-5,
    num_train_epochs=3,# TODO
)

# TODO: Change this seed when re-running your code to report the mean and std dev
set_seed(100)
num_labels = glue_tasks_num_labels[data_args.task_name]


Cloning into 'transformers'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 54602 (delta 29), reused 33 (delta 3), pack-reused 54542
Receiving objects: 100% (54602/54602), 40.79 MiB | 17.87 MiB/s, done.
Resolving deltas: 100% (38159/38159), done.
	Completed!


In [3]:
config = AutoConfig.from_pretrained(
        model_name,
        num_labels=num_labels,
        finetuning_task=data_args.task_name,
        cache_dir=".",
    )

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=".",
)

model = AutoModelWithHeads.from_pretrained(model_name, config=config)

Now we have everything set up to load our AdapterFusion setup. 

First, you need to go to adapterhub.ml and explore the different adapters available. Choose any three adapters and load the three adapters. As we don't need their prediction heads, we pass with_head=False to the loading method. Next, we add a new fusion layer that combines all the adapters we've just loaded. Finally, we add a new classification head for our target task on top.

In [4]:
# First, load the pre-trained adapters we want to fuse from Hub
from transformers.adapter_config import PfeifferConfig

model.load_adapter("nli/rte@ukp", "text_task", config=PfeifferConfig(), load_as='rte', with_head=False)
# TODO: load some more adapters. Choose the ones that you think will help RTE task after reading about the different tasks available and how big the datasets are.
model.load_adapter("nli/qnli@ukp", "text_task", config= PfeifferConfig(), load_as= 'qnli', with_head= False)
model.load_adapter("sts/mrpc@ukp", "text_task", config= PfeifferConfig(), load_as= 'mrpc', with_head= False)
# Add a fusion layer and tell the model to train fusion (freezes the rest of the weights) (here can either add the actual atsk adapter or not)
model.add_fusion([
        "rte",
        # TODO: Add your other task names here for the adapters you chose
        "qnli",
        "mrpc",
    ])

# Add a classification head for our target task
# TODO: Check the earlier notebook from Problem 5 to see how to add a classification head for your task.
model.add_classification_head("rte", num_labels= num_labels)
model.set_active_adapters([["rte", "qnli", "mrpc"]])

The last preparation step is to define and activate our adapter setup. Similar to train_adapter(), train_fusion() does two things: It freezes all weights of the model (including adapters!) except for the fusion layer and classification head. It also activates the given adapter setup to be used in very forward pass.

The syntax for the adapter setup (which is also applied to other methods such as set_active_adapters()) works as follows:

a single string is interpreted as a single adapter, 
a list of strings is interpreted as a stack of adapters,
a nested list of strings is interpreted as a fusion of adapters. Here want to do Fusion so we use a nested list as follows. 


In [5]:
adapter_setup = [
                 [
        "rte",
        # TODO: Add your other adapter names here. 
        "qnli",
        "mrpc",
    ]
]
model.train_fusion(adapter_setup)

In [6]:
# Check out your training args
print(training_args)

TrainingArguments(output_dir='./models/rte', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=True, per_device_train_batch_size=32, per_device_eval_batch_size=64, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, learning_rate=5e-05, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3, max_steps=-1, warmup_steps=0, logging_dir=None, logging_first_step=False, logging_steps=50, save_steps=1000, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_opt_level='O1', local_rank=-1, tpu_num_cores=None, tpu_metrics_debug=False)


In [7]:
train_dataset = GlueDataset(data_args, tokenizer=tokenizer)
eval_dataset = GlueDataset(data_args, tokenizer=tokenizer, mode="dev")

In [8]:
seed_values = {50, 100, 200}
best_accuracies = []
best_accuracy = -1
def compute_metrics(p: EvalPrediction):
    global best_accuracy
    preds = np.argmax(p.predictions, axis=1)
    data = glue_compute_metrics(data_args.task_name, preds, p.label_ids)
    if data['acc'] > best_accuracy:
      best_accuracy = data['acc']
    return data

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

for idx, seed_value in enumerate(seed_values):
  set_seed(seed_value)
 
  trainer.train()
  trainer.evaluate()
  best_accuracies.append(best_accuracy)
  best_accuracy = -1

print("-------Mean and Variance of best accuracies of all runs-------")
print(best_accuracies)
print(mean(best_accuracies))
print(variance(best_accuracies))

{"loss": 0.32303641356527807, "learning_rate": 3.931623931623932e-05, "epoch": 0.6410256410256411, "step": 50}



{"eval_loss": 0.9930227279663086, "eval_acc": 0.7220216606498195, "epoch": 0.6410256410256411, "step": 50}



{"loss": 0.21369579508900644, "learning_rate": 2.863247863247863e-05, "epoch": 1.282051282051282, "step": 100}



{"eval_loss": 0.8924996972084045, "eval_acc": 0.7256317689530686, "epoch": 1.282051282051282, "step": 100}
{"loss": 0.1809653341025114, "learning_rate": 1.794871794871795e-05, "epoch": 1.9230769230769231, "step": 150}



{"eval_loss": 0.9557254433631897, "eval_acc": 0.7220216606498195, "epoch": 1.9230769230769231, "step": 150}



...


{"eval_loss": 1.0039758801460266, "eval_acc": 0.7184115523465704, "epoch": 1.282051282051282, "step": 100}
{"loss": 0.1434384523704648, "learning_rate": 1.794871794871795e-05, "epoch": 1.9230769230769231, "step": 150}



{"eval_loss": 1.1079228162765502, "eval_acc": 0.7220216606498195, "epoch": 1.9230769230769231, "step": 150}



{"loss": 0.13429318312555552, "learning_rate": 7.264957264957266e-06, "epoch": 2.564102564102564, "step": 200}



{"eval_loss": 1.12352397441864, "eval_acc": 0.7292418772563177, "epoch": 2.564102564102564, "step": 200}





{"eval_loss": 1.1171754360198975, "eval_acc": 0.7292418772563177, "epoch": 3.0, "step": 234}
-------Mean and Variance of best accuracies of all runs-------
[0.7256317689530686, 0.7328519855595668, 0.7292418772563177]
0.7292418772563177
1.3032881961188047e-05
